In [1]:
import pandas as pd

In [14]:
data = pd.read_csv("twitter_training.csv", header=None, names=['id', 'topic', 'target', 'text'])
data = data[['id', 'topic', 'text', 'target']]
data = data[data['target'].isin(['Positive', 'Negative'])]
data['is_positive'] = (data['target'] == 'Positive').astype(int)
data.dropna(subset=['text', 'target'], inplace=True)

data.sample(5)

,id,topic,text,target,is_positive
62526,5119,GrandTheftAuto(GTA),Looked pretty cool too...,Positive,1
11189,13123,Xbox(Xseries),After 15 tense minutes hunting for my,Positive,1
56444,11286,TomClancysRainbowSix,"There were some angry moments, silly moments a...",Positive,1
15117,2988,Dota2,Hey @DOTA2 the way you draw the ranks is more ...,Negative,0
44213,10395,PlayerUnknownsBattlegrounds(PUBG),A,Positive,1


### Режем датасет, чтобы уложиться в лимит апишки

In [15]:
data_cutted = data.sample(n=500,
                          random_state=0,
                          ignore_index=True)[['id', 'text', 'is_positive']]

In [16]:
data_cutted['is_positive'].value_counts()

is_positive
0    258
1    242
Name: count, dtype: int64

### Подключаем psycology AI апишку и проверяем, насколько хорошо она попадает

In [91]:
from first_api import call_first_api
import random

data_cutted_with_prediction = data_cutted.copy()
data_cutted_with_prediction['random_prediction'] = pd.Series()
data_cutted_with_prediction['first_api_prediction'] = pd.Series()

for idx in data_cutted_with_prediction.index:
    if idx%20 == 0:
        print(f"Start working with row {idx}")
    
    data_cutted_with_prediction.loc[idx, 'random_prediction'] = random.random()
    data_cutted_with_prediction.loc[idx, 'first_api_prediction'] = call_first_api(data_cutted_with_prediction.loc[idx, 'text'] )['pos']

data_cutted_with_prediction

Start working with row 0
Start working with row 20
Start working with row 40
Start working with row 60
Start working with row 80
Start working with row 100
Start working with row 120
Start working with row 140
Start working with row 160
Start working with row 180
Start working with row 200
Start working with row 220
Start working with row 240
Start working with row 260
Start working with row 280
Start working with row 300
Start working with row 320
Start working with row 340
Start working with row 360
Start working with row 380
Start working with row 400
Start working with row 420
Start working with row 440
Start working with row 460
Start working with row 480


,id,text,is_positive,random_prediction,first_api_prediction
0,785,Got Pred! Flew through plt 2 to dim 1 then we ...,0,0.738767,46.7
1,13177,Where is everyone sayin how big the xbox was a...,0,0.452282,65.4
2,9645,Then Amazing.,1,0.717913,97.8
3,3260,Left Instagram because no body shows me love. ...,0,0.423604,72.0
4,12758,"Seriously, is it hard to make a good remake? G...",0,0.003479,9.7
...,...,...,...,...,...
495,7875,@EAMaddenNFL just got me stuck on using a simp...,0,0.194692,48.8
496,9728,you lie the console is very sexy<unk>,1,0.926757,26.7
497,3878,God Fuck,0,0.957366,0.1
498,8052,"Xbox: ""No good exclusives, huh?""",0,0.125826,61.8


In [92]:
from sklearn.metrics import roc_auc_score
roc_auc_random = roc_auc_score(data_cutted_with_prediction['is_positive'],
                        data_cutted_with_prediction['random_prediction'])

roc_auc_first_api = roc_auc_score(data_cutted_with_prediction['is_positive'],
                        data_cutted_with_prediction['first_api_prediction'])

print(roc_auc_random, roc_auc_first_api)

0.47262797104234733 0.7803510795054135


### Подключаем вторую апишку и проверяем, насколько хорошо она попадает

In [18]:
from second_api import call_second_api
import random

data_cutted_with_prediction = data_cutted.copy()
data_cutted_with_prediction['random_prediction'] = pd.Series()
data_cutted_with_prediction['second_api_prediction'] = pd.Series()

for idx in data_cutted_with_prediction.index:
    if idx%10 == 0:
        print(f"Start working with row {idx}")

    ans = call_second_api(data_cutted_with_prediction.loc[idx, 'text'] )
    
    data_cutted_with_prediction.loc[idx, 'random_prediction'] = random.random()
    data_cutted_with_prediction.loc[idx, 'second_api_prediction'] = ans['pos'] + ans['neu']/2

data_cutted_with_prediction

Start working with row 0
Start working with row 10
Start working with row 20
Start working with row 30
Start working with row 40
Start working with row 50
Start working with row 60
Start working with row 70
Start working with row 80
Start working with row 90
Start working with row 100
Start working with row 110
Start working with row 120
Start working with row 130
Start working with row 140
Start working with row 150
Start working with row 160
Start working with row 170
Start working with row 180
Start working with row 190
Start working with row 200
Start working with row 210
Start working with row 220
Start working with row 230
Start working with row 240
Start working with row 250
Start working with row 260
Start working with row 270
Start working with row 280
Start working with row 290
Start working with row 300
Start working with row 310
Start working with row 320
Start working with row 330
Start working with row 340
Start working with row 350
Start working with row 360
Start workin

,id,text,is_positive,random_prediction,second_api_prediction
0,785,Got Pred! Flew through plt 2 to dim 1 then we ...,0,0.344914,42.05
1,13177,Where is everyone sayin how big the xbox was a...,0,0.817499,42.25
2,9645,Then Amazing.,1,0.742359,89.6
3,3260,Left Instagram because no body shows me love. ...,0,0.311747,62.0
4,12758,"Seriously, is it hard to make a good remake? G...",0,0.707049,53.35
...,...,...,...,...,...
495,7875,@EAMaddenNFL just got me stuck on using a simp...,0,0.264657,46.9
496,9728,you lie the console is very sexy<unk>,1,0.799545,50.0
497,3878,God Fuck,0,0.338058,37.5
498,8052,"Xbox: ""No good exclusives, huh?""",0,0.7757,54.3


In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_random = roc_auc_score(data_cutted_with_prediction['is_positive'],
                        data_cutted_with_prediction['random_prediction'])

roc_auc_second_api = roc_auc_score(data_cutted_with_prediction['is_positive'],
                        data_cutted_with_prediction['second_api_prediction'])

print(roc_auc_random, roc_auc_second_api)

0.5212377474533922 0.7961592670894997


In [22]:
data_cutted_with_prediction.sample(10)

,id,text,is_positive,random_prediction,second_api_prediction
427,2754,my,1,0.799216,50.0
448,8409,@ Ron2K @ LD2K @ Beluba @ nba2k I need answers...,0,0.810778,50.0
87,457,That would be awesome! Love the ultimate idea,1,0.39716,79.45
98,9005,This shit boutta make me build a PC,0,0.17094,31.25
252,6494,Looking through pc specs: yeah this CAN run fo...,0,0.895933,45.95
353,2440,This is shitty. I get that profit - wise it re...,0,0.380506,58.35
44,1315,I really hate how balance in the team separate...,0,0.069961,41.6
193,11671,@verizon. sorry I cancelled your ALL services....,0,0.840272,48.15
137,10896,about everyone tips how you improve the GR Bre...,0,0.69894,55.2
181,1850,Fuck sleep. Watch the trailer for the Cold War...,1,0.791798,33.5
